In [16]:
import os, sys
from pathlib import Path
from typing import Dict, Tuple, List
import spacy

HOME = os.getcwd()
DATA_FOLDER = os.path.join(Path(HOME).parent, 'data')
data_path = os.path.join(DATA_FOLDER, 'filtered.tsv')

In [17]:
current = HOME 
while 'src' not in os.listdir(current):
    current = Path(current).parent

PARENT_DIR = current

sys.path.append(str(current))
sys.path.append(os.path.join(str(current), 'data_analysis'))
sys.path.append(os.path.join(str(current), 'evaluation'))
sys.path.append(os.path.join(str(current), 'text_processing'))

In [18]:
# let's first fix the data 
import src.text_processing.preprocess as pr
import src.data_preparation.prepare_data as prd 
# fixed_data = prd.fix_initial_data(data_path)
data = prd.prepare_all_data(fixed_data_file=os.path.join(DATA_FOLDER, 'fixed.csv'), save=False)

In [21]:
sample = data.select(range(10 ** 4))
def process_text(text: str) -> str:
    return pr.no_extra_spaces(pr.no_extra_chars(pr.to_lower(text)))

def process_batch(batch: Dict, nlp):
    return dict([(k, [process_text(t) for t in v]) for k, v in ({"source": pr.uniform_ne_batched(batch['source'], nlp), 
                                                                 "target": pr.uniform_ne_batched(batch['target'], nlp)}).items()]) 

# load the nlp object in advnace
# nlp = spacy.load("en_core_web_sm")
# processed_data = data.map(lambda b: process_batch(b, nlp), batched=True)
# # save the data
# processed_data.to_csv(os.path.join(DATA_FOLDER, 'all_data_processed.csv'), index=False)

In [22]:
from datasets import load_dataset
processed_data = load_dataset("csv", data_files=os.path.join(DATA_FOLDER, "all_data_processed.csv"), split='train')
# make sure all the data is 'string' format
processed_data = processed_data.filter(lambda s: (isinstance(s['source'], str) and isinstance(s['target'], str)))

Filter: 100%|██████████| 597521/597521 [00:01<00:00, 547500.16 examples/s]


In [23]:
# let's take a small smaple of 10000 rows
from src.toxicity_scores import n_grams as ng
import importlib
importlib.reload(ng)
# sample = processed_data.select(range(10 ** 4))
uni, bi = ng.build_unigram_counter(processed_data)
ignore_map = ng.build_ignore_toxic_map(0.1, pr.standard_stop_words())
# ignore_map

Map: 100%|██████████| 597519/597519 [01:24<00:00, 7064.64 examples/s]


8021 non alpha tokens


In [24]:
u, b = dict([(k, (v["source"] + 1) / (v["target"] + 1)) for k, v in uni.items()]) , dict([(k, (v["source"] + 1) / (v["target"] + 1)) for k, v in bi.items()]) 

In [1]:
importlib.reload(ng)
s = "Are you a nice person ?"
ids = ng._prepare_sentence(s, pr.standard_stop_words())
ng.indices_toxicity_score(indices=ids, ignore_map=ignore_map, uni_gram=u, bi_gram=b, default_toxicity=0.1)

NameError: name 'importlib' is not defined

In [ ]:
from transformers import AutoTokenizer, DistilBertModel
bert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
# bert_tokenizer.convert_ids_to_tokens([6583, 2131])

['na', 'get']

In [ ]:
import pickle
with open(os.path.join(PARENT_DIR, 'src', 'toxicity_scores', 'uni_gram.pk'), 'rb') as f:
    counter_loaded = pickle.load(f) 